# Prepare environment
Load libraries, etc.

In [1]:
%pip install -q transformers librosa datasets==2.14.6 evaluate jiwer gradio bitsandbytes==0.37 accelerate
%pip install -q git+https://github.com/huggingface/peft.git@main

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import notebook_login

notebook_login() #huggingface-cli login workaround

In [27]:
from datasets import load_dataset, DatasetDict
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# Load dataset

In [7]:
sd_qa = DatasetDict()

sd_qa["dev"] = load_dataset("WillHeld/SD-QA", split="dev", use_auth_token=True)
sd_qa["test"] = load_dataset("WillHeld/SD-QA", split="test", use_auth_token=True)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating dev split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1031 [00:00<?, ? examples/s]

/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [10]:
print(sd_qa)

DatasetDict({
    dev: Dataset({
        features: ['id', 'aus', 'gbr', 'ind_n', 'ind_s', 'irl', 'kenya', 'nga', 'nzl', 'phl', 'usa', 'zaf', 'answers', 'question'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'aus', 'gbr', 'ind_n', 'ind_s', 'irl', 'kenya', 'nga', 'nzl', 'phl', 'usa', 'zaf', 'answers', 'question'],
        num_rows: 1031
    })
})


In [20]:
print(sd_qa['dev'])

Dataset({
    features: ['id', 'aus', 'gbr', 'ind_n', 'ind_s', 'irl', 'kenya', 'nga', 'nzl', 'phl', 'usa', 'zaf', 'answers', 'question'],
    num_rows: 1000
})


# Prepare data

In [33]:
# select only target and source dialect
target_dialect = 'usa'
source_dialect = 'aus'
sd_qa = sd_qa.select_columns(['id', source_dialect, target_dialect])
print(sd_qa['dev'][0])

{'id': '-1008642825401516622', 'aus': {'path': None, 'array': array([-0.00289917, -0.00250244, -0.00326538, ...,  0.00161743,
        0.0017395 ,  0.00253296]), 'sampling_rate': 16000}, 'usa': {'path': None, 'array': array([0.        , 0.        , 0.        , ..., 0.00201416, 0.00259399,
       0.00262451]), 'sampling_rate': 16000}}


In [29]:
# load whisper feature extractor, tokenizer, processor
model_path = "openai/whisper-base"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
task = "transcribe"
tokenizer = WhisperTokenizer.from_pretrained(model_path, task=task)
processor = WhisperProcessor.from_pretrained(model_path, task=task)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [38]:
# define prepare dataset function to extract audio features
def prepare_dataset(batch):

    # compute log-Mel input features from input audio array
    batch["source_input_features"] = feature_extractor(batch[source_dialect]["array"], sampling_rate=batch[source_dialect]["sampling_rate"]).input_features[0]
    batch["target_input_features"] = feature_extractor(batch[target_dialect]["array"], sampling_rate=batch[target_dialect]["sampling_rate"]).input_features[0]

    return batch

In [39]:
# map feature extracter to sd_qa
sd_qa = sd_qa.map(prepare_dataset, remove_columns=sd_qa.column_names["dev"], num_proc=2)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=2):   0%|          | 0/1031 [00:00<?, ? examples/s]

/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


In [40]:
print(sd_qa['dev'])

Dataset({
    features: ['source_input_features', 'target_input_features'],
    num_rows: 1000
})


# Training 

# Evaluation

# Inference